<style>
h1 {
    color: blue;
    font-weight: bold;
}
</style>

# TP4 — Implémentation d’un Système Expert en Python


# Khadija Benhamida IID3


<h3 style="color:#0078B8;">Partie 1 : Modélisation des faits et règles</h3>


In [7]:

fait_fievre = "fievre"
fait_toux = "toux"
fait_douleurs_thoraciques = "douleurs_thoraciques"
fait_essoufflement = "essoufflement"

R1 = {
    "id": "R1",
    "conditions": ["fievre", "toux"],
    "conclusion": "grippe"
}

R2 = {
    "id": "R2",
    "conditions": ["grippe", "douleurs_thoraciques"],
    "conclusion": "infection_respiratoire"
}

R3 = {
    "id": "R3",
    "conditions": ["infection_respiratoire", "essoufflement"],
    "conclusion": "hospitalisation_conseillée"
}

base_de_regles = [R1, R2, R3]

if __name__ == "__main__":
    print("Faits d'exemple :")
    print(" -", fait_fievre)
    print(" -", fait_toux)
    print(" -", fait_douleurs_thoraciques)
    print(" -", fait_essoufflement)
    print()
    print("Base de règles :")
    for r in base_de_regles:
        print(f" {r['id']}: SI {r['conditions']} ALORS {r['conclusion']}")


Faits d'exemple :
 - fievre
 - toux
 - douleurs_thoraciques
 - essoufflement

Base de règles :
 R1: SI ['fievre', 'toux'] ALORS grippe
 R2: SI ['grippe', 'douleurs_thoraciques'] ALORS infection_respiratoire
 R3: SI ['infection_respiratoire', 'essoufflement'] ALORS hospitalisation_conseillée



<h3 style="color:#0078B8;">Partie 2 : chainage_avant</h3>


In [9]:

R1 = {"id": "R1", "conditions": ["fievre", "toux"], "conclusion": "grippe"}
R2 = {"id": "R2", "conditions": ["grippe", "douleurs_thoraciques"], "conclusion": "infection_respiratoire"}
R3 = {"id": "R3", "conditions": ["infection_respiratoire", "essoufflement"], "conclusion": "hospitalisation_conseillée"}

regles = [R1, R2, R3]

def chainage_avant(faits_initiaux, regles, trace=True):
    faits = set(faits_initiaux)
    
    changement = True  # indicateur de point fixe

    while changement:
        changement = False
        for regle in regles:
            conds = regle["conditions"]
            concl = regle["conclusion"]
            rid = regle["id"]

            if all(c in faits for c in conds) and concl not in faits:
                faits.add(concl)
                changement = True

                if trace:
                    print(f"Règle appliquée ({rid}) : SI {conds} ALORS {concl}")

    return list(faits)


if __name__ == "__main__":
    print("\n======================")
    print("      CAS 1")
    print("======================")
    faits1 = ["fievre", "toux"]
    print("Faits initiaux :", faits1)
    final1 = chainage_avant(faits1, regles, trace=True)
    print("Faits finaux   :", sorted(final1))
    print("Conclusion : Le système déduit → grippe\n")

    print("======================")
    print("      CAS 2")
    print("======================")
    faits2 = ["fievre", "toux", "douleurs_thoraciques", "essoufflement"]
    print("Faits initiaux :", faits2)
    final2 = chainage_avant(faits2, regles, trace=True)
    print("Faits finaux   :", sorted(final2))
    print("Conclusion : Le système déduit toute la chaîne → grippe → infection_respiratoire → hospitalisation_conseillée\n")

    print("======================")
    print("      CAS 3")
    print("======================")
    faits3 = ["fievre"]
    print("Faits initiaux :", faits3)
    final3 = chainage_avant(faits3, regles, trace=True)
    print("Faits finaux   :", sorted(final3))
    print("Conclusion : Aucune règle ne se déclenche\n")



      CAS 1
Faits initiaux : ['fievre', 'toux']
Règle appliquée (R1) : SI ['fievre', 'toux'] ALORS grippe
Faits finaux   : ['fievre', 'grippe', 'toux']
Conclusion : Le système déduit → grippe

      CAS 2
Faits initiaux : ['fievre', 'toux', 'douleurs_thoraciques', 'essoufflement']
Règle appliquée (R1) : SI ['fievre', 'toux'] ALORS grippe
Règle appliquée (R2) : SI ['grippe', 'douleurs_thoraciques'] ALORS infection_respiratoire
Règle appliquée (R3) : SI ['infection_respiratoire', 'essoufflement'] ALORS hospitalisation_conseillée
Faits finaux   : ['douleurs_thoraciques', 'essoufflement', 'fievre', 'grippe', 'hospitalisation_conseillée', 'infection_respiratoire', 'toux']
Conclusion : Le système déduit toute la chaîne → grippe → infection_respiratoire → hospitalisation_conseillée

      CAS 3
Faits initiaux : ['fievre']
Faits finaux   : ['fievre']
Conclusion : Aucune règle ne se déclenche



<h3 style="color:#0078B8;">Partie 3 : Interaction avec le système</h3>


In [10]:

R1 = {"id": "R1", "conditions": ["fievre", "toux"], "conclusion": "grippe"}
R2 = {"id": "R2", "conditions": ["grippe", "douleurs_thoraciques"], "conclusion": "infection_respiratoire"}
R3 = {"id": "R3", "conditions": ["infection_respiratoire", "essoufflement"], "conclusion": "hospitalisation_conseillée"}

regles = [R1, R2, R3]


def chainage_avant(faits_initiaux, regles, trace=False):
    faits = set(faits_initiaux)
    changement = True

    while changement:
        changement = False
        for regle in regles:
            conds = regle["conditions"]
            conc = regle["conclusion"]

            if all(c in faits for c in conds) and conc not in faits:
                faits.add(conc)
                changement = True
                if trace:
                    print(f"Règle appliquée ({regle['id']}) : SI {conds} ALORS {conc}")

    return list(faits)


def peut_on_deduire(fait_cible, faits_initiaux, regles, trace=True):
 
    faits_finaux = chainage_avant(faits_initiaux, regles, trace=trace)

    if fait_cible in faits_finaux:
        print(f"Oui, {fait_cible} est déduite.")
        return True
    else:
        print(f"Non, le système ne peut pas conclure {fait_cible}.")
        return False


if __name__ == "__main__":
    print("=== Test 1 ===")
    peut_on_deduire("grippe", ["fievre", "toux"], regles)

    print("\n=== Test 2 ===")
    peut_on_deduire("infection_respiratoire",
                    ["fievre", "toux", "douleurs_thoraciques", "essoufflement"],
                    regles)

    print("\n=== Test 3 ===")
    peut_on_deduire("infection_respiratoire", ["fievre"], regles)


=== Test 1 ===
Règle appliquée (R1) : SI ['fievre', 'toux'] ALORS grippe
Oui, grippe est déduite.

=== Test 2 ===
Règle appliquée (R1) : SI ['fievre', 'toux'] ALORS grippe
Règle appliquée (R2) : SI ['grippe', 'douleurs_thoraciques'] ALORS infection_respiratoire
Règle appliquée (R3) : SI ['infection_respiratoire', 'essoufflement'] ALORS hospitalisation_conseillée
Oui, infection_respiratoire est déduite.

=== Test 3 ===
Non, le système ne peut pas conclure infection_respiratoire.
